# 코드설명(임시)
- GAN 중간값 pca하는 코드

In [3]:
import tqdm

def nop(it, *a, **k):
    return it

real_tqdm = tqdm.tqdm
tqdm.tqdm = nop

import glob
import pickle
import os
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

np.bool = np.bool_

pca_save_dir = "./pca_pkl"
os.makedirs(pca_save_dir, exist_ok=True)

# PCA for zattr[0~7]
- 층마다의 데이터 비율: [1.0, 1.0, 1.0, 1.0, 0.9, 0.8, 0.5, 0.5]
- PCA 컴포넌트 128개 제한

In [4]:
for attr_id, data_sRate in zip(range(8), [1.0, 1.0, 1.0, 1.0, 0.9, 0.8, 0.5, 0.5]):
    np.random.seed(7777)
    emb_array = []

    emb_file_list = glob.glob("./examples/z_embeds/VggFace2-crop/*.pkl")

    for file_path in real_tqdm(emb_file_list):
        if np.random.random() < data_sRate:
            with open(file_path, "rb") as file:
                emb_array.append(pickle.load(file)[attr_id].reshape([-1]))
    emb_array = np.array(emb_array)

    pca = PCA(n_components=128, svd_solver="auto")

    pca.fit(emb_array)
    print(np.sum(pca.explained_variance_ratio_))
    with open(f"{pca_save_dir}/Altered_zattr{attr_id}PCA.pkl", "wb") as file:
        pickle.dump(pca, file)

    max_pca_array = -np.inf
    min_pca_array = np.inf

    for i in real_tqdm(range(emb_array.shape[0])):
        tmp = emb_array[i:i+1]
        pca_array = pca.transform(tmp)
        max_pca_array = np.maximum(max_pca_array, pca_array)
        min_pca_array = np.minimum(min_pca_array, pca_array)

    pca_minmax = dict({"min": min_pca_array[0], "max": max_pca_array[0]})

    with open(f"{pca_save_dir}/Altered_zattr{attr_id}PCAMinMax.pkl", "wb") as file:
        pickle.dump(pca_minmax, file)
    
    del pca, emb_array, pca_array, max_pca_array, min_pca_array, pca_minmax, tmp

0.999868119409415
0.9411023773789768
0.9221101978155233
0.7781026173732422
0.7282376222433978
0.7007684418879786
0.7398766752827983
0.7672928256499298


# PCA Again
- zattr[0~7] 각각에 대하여 PCA화 된 것을 다시 PCA함
- 컴포넌트 128개

In [5]:

pca_list = []
for attr_id in range(8):
    with open(f"{pca_save_dir}/Altered_zattr{attr_id}PCA.pkl", "rb") as file:
        pca_list.append(pickle.load(file))
pcaed_emb_array = []
emb_file_list = glob.glob("./examples/z_embeds/VggFace2-crop/*.pkl")
for file_path in real_tqdm(emb_file_list):
    with open(file_path, "rb") as file:
        tmp = pickle.load(file)
    for attr_id in range(8):
        tmp[attr_id] = pca_list[attr_id].transform(tmp[attr_id].reshape(1, -1))
    pcaed_emb_array.append(np.concatenate(tmp, axis=1))

pcaed_emb_array = np.concatenate(pcaed_emb_array, axis=0)
pcapca = PCA(n_components=128, svd_solver="auto")
pcapca.fit(pcaed_emb_array)
print(np.sum(pcapca.explained_variance_ratio_))
with open(f"{pca_save_dir}/Altered_zattr_doublePCA.pkl", "wb") as file:
    pickle.dump(pcapca, file)

max_pca_array = -np.inf
min_pca_array = np.inf

for i in real_tqdm(range(pcaed_emb_array.shape[0])):
    tmp = pcaed_emb_array[i:i+1]
    pcapca_array = pcapca.transform(tmp)
    max_pca_array = np.maximum(max_pca_array, pcapca_array)
    min_pca_array = np.minimum(min_pca_array, pcapca_array)

pcapca_minmax = dict({"min": min_pca_array[0], "max": max_pca_array[0]})

with open(f"{pca_save_dir}/Altered_zattr_doublePCAMinMax.pkl", "wb") as file:
    pickle.dump(pcapca_minmax, file)


0.9745815360463025


In [8]:
pcaed_emb_array.shape

(17262, 1, 1024)